In [1]:
%%capture
!pip install torch 
!pip install torchvision

In [3]:
import io
import torchvision
import pathlib
import torch
import os
from torch.utils.data import Dataset
from datasets import load_dataset
from PIL import Image
from pathlib import Path
from typing import Tuple, List, Dict

C:\Users\Naya\anaconda3\envs\colab\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
dataset = load_dataset("C:\\Users\\Naya\\.cache\\huggingface\\datasets\\pantelism___cats-vs-dogs")

Generating train split: 4000 examples [00:00, 292903.44 examples/s]


In [7]:
dataset = dataset['train'].train_test_split(test_size=0.3, seed=42)

In [9]:
path = "C:\\datasets\\catsdogs"
for i, example in enumerate(dataset['train']):
    if 'image' in example:
        image = example['image']
        if example['label'] == 0: 
            image.save(f'{path}\\train\\cats\\image_{i}.jpg')
        else:
            image.save(f'{path}\\train\\dogs\\image_{i}.jpg')

for i, example in enumerate(dataset['test']):
    if 'image' in example:
        image = example['image']
        if example['label'] == 0: 
            image.save(f'{path}\\test\\cats\\image_{i}.jpg')
        else:
            image.save(f'{path}\\test\\dogs\\image_{i}.jpg')

In [11]:
def find_classes(directory: str) -> Tuple[List[str], Dict[str, int]]:
    classes = sorted(entry.name for entry in os.scandir(directory) if entry.is_dir())
    
    if not classes:
        raise FileNotFoundError(f"Couldn't find any classes in {directory}.")
        
    class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
    return classes, class_to_idx

In [13]:
class ImageFolderCustom(Dataset):
    def __init__(self, targ_dir: str, transform=None) -> None:
        self.paths = list(pathlib.Path(targ_dir).glob("*/*.jpg"))
        self.transform = transform
        self.classes, self.class_to_idx = find_classes(targ_dir)

    def load_image(self, index: int) -> Image.Image:
        image_path = self.paths[index]
        return Image.open(image_path) 
    
    def __len__(self) -> int:
        return len(self.paths)
    
    def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
        img = self.load_image(index)
        class_name  = self.paths[index].parent.name
        class_idx = self.class_to_idx[class_name]

        if self.transform:
            return self.transform(img), class_idx
        else:
            return img, class_idx

In [15]:
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((64, 64)),
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.ToTensor()
])

test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((64, 64)),
    torchvision.transforms.ToTensor()
])

In [17]:
train_data_custom = ImageFolderCustom(targ_dir = f"{path}\\train\\",
                                     transform=train_transforms)

test_data_custom = ImageFolderCustom(targ_dir = f"{path}\\test\\",
                                     transform=test_transforms)


In [19]:
train_data_custom, test_data_custom

(<__main__.ImageFolderCustom at 0x1f12937a7a0>,
 <__main__.ImageFolderCustom at 0x1f112cdf970>)